In [1]:
!pip install bitstring

In [2]:
from bitstring import BitArray
import numpy as np
import random

In [3]:
def isBoardFinished(boardState):
    return boardState.bin=='111111111111'

In [4]:
def transition(currentState,currentMove):
    temp=currentState
    temp=temp[:currentMove-1]+'1'+temp[currentMove:]
#     print("Transition from state",currentState,"Is ",temp)
    return temp

#print(transition(BitArray(uint=10,length=12),11))

In [5]:
def numberOfNewBoxFormed(box,newState):
    count=0
    for i in range(4):
        if i==0 and box[i]==0:
            if newState[0]=='1' and newState[2]=='1' and newState[6]=='1' and newState[7]=='1':
                count+=1
                box[i]=1
        if i==1 and box[i]==0:
            if newState[1]=='1' and newState[3]=='1' and newState[7]=='1' and newState[8]=='1':
                count+=1
                box[i]=1
        if i==2 and box[i]==0:
            if newState[2]=='1' and newState[4]=='1' and newState[9]=='1' and newState[10]=='1':
                count+=1
                box[i]=1
        if i==3 and box[i]==0:
            if newState[3]=='1' and newState[5]=='1' and newState[10]=='1' and newState[11]=='1':
                count+=1
                box[i]=1
    return count
        
                

In [6]:
'''
3x3 dots and boxes 
4 boxes 
12 lines to choose from for each move
2 power 12 game states for the agent to learn from
DRAW / TIE is available in this version of the game
Reward 100 and Penalty -100
'''
DISCOUNT_RATE =0.8
LEARNING_RATE =0.2


In [7]:
def learner():
    initialiseQTable()
    gameCount=1
    choiceOfMove=['random','simple','qlearner']
    while gameCount<10000:
        print("GAME ",gameCount)
        if gameCount<10000:
            playGame(choiceOfMove[0],False)
        elif gameCount<50000:
            playGame(choiceOfMove[1],False)
        else:
            playGame(choiceOfMove[2],False)
        gameCount+=1
    # save the Qtable
#     playGameWithHuman()
    
        
        

In [8]:
QTable=dict()
VISIT_TABLE=dict()
def initialiseQTable():
    global QTable
    global VISIT_TABLE
    for i in range(4096):
        currentState=BitArray(uint=i, length=12)
        actionQvalues=dict()
        # find possible actions
        bitString=currentState.bin
        for index in range(len(bitString)):
            if bitString[index]=='0':
                actionQvalues[index+1]=0
        QTable[currentState.bin]=actionQvalues
        VISIT_TABLE[currentState.bin]=0
        QTable['111111111111']={0:0}
initialiseQTable()

                
        
    

In [9]:
def bestActionForState(state):
    maxQval=-1 #negative inf
    bestAction=0
    for action,qval in QTable[state].items():
        if qval>maxQval:
            maxQval=qval
            bestAction=action
    return bestAction
def updateQTable(memoryOfPlayerMoves):
    global QTable
    memoryOfPlayerMoves=memoryOfPlayerMoves[::-1] #reverse chronological order
    for state,action,nextState,reward in memoryOfPlayerMoves:
        maxQValForNextState=QTable[nextState][bestActionForState(nextState)]
        existingValue=(1-LEARNING_RATE)*QTable[state][action]
        updateValue=LEARNING_RATE*(reward+DISCOUNT_RATE*maxQValForNextState)
        QTable[state][action]=existingValue+updateValue
        VISIT_TABLE[state]+=1
    
        

    
    
    
    
    

In [10]:
def printTable(currentState):
    count=1
    m=7
    for i in range(3):
        for j in range(3):
            print(".\t", end=" ")
            if j<2:
                if currentState[count-1]=='0':
                    print(str(count)+"\t",end=" ") 
                else:
                    print("--\t",end=" ")
                count+=1
                
        print("\n")
        if i!=2:
            for k in range(3):
                if currentState[m-1]=='0':
                    print(str(m)+"\t\t",end=" ")
                else:
                    print("|\t\t",end=" ")
                m+=1
        print("\n")
printTable('111111111111')

.	 --	 .	 --	 .	 

|		 |		 |		 

.	 --	 .	 --	 .	 

|		 |		 |		 

.	 --	 .	 --	 .	 





In [11]:

def MakeMove(currentState,choice):
    # possible actions
    possibleActions=list(QTable[currentState].keys())
#     print(possibleActions)
    #choiceOfMove=['random','simple','qlearner']
    if choice=='random':
        return possibleActions[random.randint(0,len(possibleActions)-1)]
        
    if choice=='simple':
        minCount=1000000
        action=0
        for action in PossibleActions:
            if VISIT_TABLE[transition(currentState,action)]<minCount:
                minCount=VISIT_TABLE[transition(currentState,action)]
                bestAction=action     
        return bestAction
    if choice=='qlearner':
        return bestActionForState(currentState)
        

    

    
 

In [12]:
def playGame(choice,isHuman):
    #Game init
    player=["p1","p2"]
    if isHuman:
        player[1]='human'
    currentPlayer="p1"
    box=[0,0,0,0]
    currentState=BitArray(uint=0, length=12)
    memory={player[0]:[],player[1]:[]}
    score={player[0]:0,player[1]:0}
    while isBoardFinished(currentState)==False:
#         print("Playing game")
        if(currentPlayer!="human"):
            currentMove=MakeMove(currentState.bin,choice)
        else:
            currentMove=int(input("Enter the line number:"))
        newState=transition(currentState.bin,currentMove) 
        memory[currentPlayer].append([currentState.bin,currentMove,newState,0])
        numberOfBoxes=numberOfNewBoxFormed(box,newState)
        if numberOfBoxes>0:#Update this......:)
            score[currentPlayer]+=numberOfBoxes*2
        else:
            currentPlayer=player[1-player.index(currentPlayer)]

        currentState=BitArray(uint=int(newState,2),length=12)
        if player[1]=='human':
            
            printTable(currentState.bin)
            print("Score of ",player[0]," is ",score[player[0]])
            print("Score of ",player[1]," is ",score[player[1]])
            print("Next turn for ",currentPlayer)

    #find winner 
    result=""
    if score[player[0]]==score[player[1]]:
        result='TIE'
        print(result)
    else:
        result=max(score,key=score.get)
        print("Winner is ",result)
    #print(memory[player[0]])
    if result=='TIE':
        memory[player[0]][:-1][0][-1]=50
        memory[player[1]][:-1][0][-1]=50
    else: 
        memory[result][:-1][0][-1]=100
        memory[player[1-player.index(result)]][:-1][0][-1]=0    
    #put reward and penalties in last row of the memory table
    updateQTable(memory[player[0]])
    updateQTable(memory[player[1]])





In [ ]:
learner()
playGame('qlearner',True)

GAME  1
Winner is  p2
GAME  2
Winner is  p1
GAME  3
Winner is  p2
GAME  4
Winner is  p2
GAME  5
Winner is  p1
GAME  6
Winner is  p1
GAME  7
Winner is  p1
GAME  8
Winner is  p2
GAME  9
Winner is  p1
GAME  10
Winner is  p2
GAME  11
TIE
GAME  12
Winner is  p2
GAME  13
Winner is  p2
GAME  14
Winner is  p1
GAME  15
Winner is  p1
GAME  16
Winner is  p1
GAME  17
Winner is  p1
GAME  18
Winner is  p1
GAME  19
Winner is  p1
GAME  20
Winner is  p2
GAME  21
Winner is  p2
GAME  22
Winner is  p1
GAME  23
TIE
GAME  24
TIE
GAME  25
Winner is  p2
GAME  26
Winner is  p2
GAME  27
Winner is  p2
GAME  28
Winner is  p1
GAME  29
Winner is  p2
GAME  30
Winner is  p1
GAME  31
Winner is  p2
GAME  32
Winner is  p2
GAME  33
Winner is  p1
GAME  34
Winner is  p1
GAME  35
Winner is  p2
GAME  36
Winner is  p1
GAME  37
Winner is  p1
GAME  38
Winner is  p1
GAME  39
Winner is  p1
GAME  40
Winner is  p2
GAME  41
Winner is  p2
GAME  42
Winner is  p1
GAME  43
Winner is  p2
GAME  44
Winner is  p2
GAME  45
Winner is  p2
GAME

Enter the line number: 1


.	 --	 .	 --	 .	 

7		 8		 9		 

.	 3	 .	 4	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  0
Next turn for  p1
.	 --	 .	 --	 .	 

7		 8		 9		 

.	 --	 .	 4	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  0
Next turn for  human


Enter the line number: 7


.	 --	 .	 --	 .	 

|		 8		 9		 

.	 --	 .	 4	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  0
Next turn for  p1
.	 --	 .	 --	 .	 

|		 8		 9		 

.	 --	 .	 --	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  0
Next turn for  human


Enter the line number: 8


.	 --	 .	 --	 .	 

|		 |		 9		 

.	 --	 .	 --	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  2
Next turn for  human


Enter the line number: 9


.	 --	 .	 --	 .	 

|		 |		 |		 

.	 --	 .	 --	 .	 

10		 11		 12		 

.	 5	 .	 6	 .	 



Score of  p1  is  0
Score of  human  is  4
Next turn for  human
